In [1]:
import pandas as pd
import plotly.graph_objects as go
import os

### COMBINE SUBCLUSTER FILES

In [5]:
dir = r"data\20230416\subclusters"
df = pd.DataFrame()
for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        temp_df = pd.read_csv(f)
        temp_df.rename(columns={'SSN Cluster Number':'Subcluster Number'}, inplace=True)
        cluster_number_long = filename.split("_")[3].split('.')[0]
        cluster_number = cluster_number_long.replace("Cluster", "")
        temp_df['SSN Cluster Number'] = cluster_number
        df = pd.concat([df, temp_df], axis=0)

In [2]:
dir = r"data\20230416\neighbors"
df = pd.DataFrame()
for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        temp_df = pd.read_csv(f)
        temp_df.rename(columns={'cluster_num':'Subcluster Number'}, inplace=True)
        cluster_number_long = filename.split("_")[1].split('.')[0]
        cluster_number = cluster_number_long.replace("cluster", "")
        temp_df['SSN Cluster Number'] = cluster_number
        df = pd.concat([df, temp_df], axis=0)

In [6]:
df.iloc[0]

# of Pfam Neighbors                                                                           5.0
# of Queries with Pfam Neighbors                                                              5.0
# of Sequences in SSN Cluster                                                                  18
# of Sequences in SSN Cluster with Neighbors                                                   14
Average Distance                                                                             4.59
Co-occurrence                                                                                0.35
Co-occurrence Ratio                                                                          5/14
Hub Average and Median Distance                                                               NaN
Hub Co-occurrence and Ratio                                                                   NaN
Hub Pfam Neighbors                                                                            NaN
Hub Queries with Pfa

In [7]:
df.to_csv(r'data/subclusters.csv')

### assign tags for regulator and transporter

In [3]:
attributes = pd.read_csv(r'data\attributes.csv')
neighbors = pd.read_csv(r'data\neighbors.csv')

In [6]:
neighbors['transporter'] = neighbors['desc'].apply(lambda x: 'transport' in x if type(x)==str else False)
neighbors['regulator'] = neighbors['desc'].apply(lambda x: 'regulat' in x if type(x)==str else False)
neighbors['EamA'] = neighbors['desc'].apply(lambda x: 'EamA' in x if type(x)==str else False)

In [20]:
neighbors.to_csv(r'data\neighbors.csv')

### get pairs of neighbors

In [3]:
neighbors = pd.read_csv('./data/neighbors.csv')
attributes = pd.read_csv('./data/attributes.csv')
subcluster_df = pd.read_csv('./data/AlaX_10neighbors.csv')

THRESHOLD = 10
# flag_proteins = attributes.loc[attributes['SSN Cluster Number']==SSN].index
SSN=5

In [4]:
flag_proteins = attributes.loc[attributes['SSN Cluster Number']==SSN].index
subcluster_df = subcluster_df.loc[subcluster_df['SSN Cluster Number']==SSN]

count_neighbors = {}
dict_flag_proteins = {}
filtered_neighbors = pd.DataFrame()
for flag in flag_proteins:
    taxon = attributes.loc[flag, 'taxon_id']
    flag_num = attributes.loc[flag, 'num']
    flag_neighbors = neighbors.loc[(neighbors['taxon_id']==taxon)&(neighbors['num']>=(flag_num-THRESHOLD))&(neighbors['num']<=(flag_num+THRESHOLD))&(neighbors['family']!='none')]
    dict_flag_proteins = flag_neighbors['family_desc'].unique()
    filtered_neighbors = pd.concat([filtered_neighbors, flag_neighbors], axis=0)
    for neighbor in flag_neighbors['family_desc'].unique():
        count_neighbors[neighbor] = count_neighbors[neighbor]+1 if neighbor in count_neighbors.keys() else 1

neighbor_list = filtered_neighbors.groupby('taxon_id')['family_desc'].apply(list).reset_index(name="family_desc")

In [5]:
top_neighbors = dict(sorted(count_neighbors.items(), key=lambda x:x[1], reverse=True)[:5])

In [26]:
count_secondaryneighbors = pd.DataFrame(columns=['index'])
for neighbor in top_neighbors.keys():
    neighbor_list[neighbor+'_presence'] = neighbor_list.apply(lambda x: neighbor in x['family_desc'], axis=1)
    temp = neighbor_list.loc[neighbor_list[neighbor+'_presence']].explode('family_desc')
    temp_secondary = temp['family_desc'].value_counts(ascending=False).reset_index(name=neighbor).iloc[:10,:]
    # count_secondaryneighbors = pd.concat([count_secondaryneighbors, temp_secondary[neighbor]], axis=1)
    count_secondaryneighbors = pd.merge(count_secondaryneighbors, temp_secondary, how="outer", on='index')
    # count_secondaryneighbors = count_secondaryneighbors.join(temp_secondary)
count_secondaryneighbors = count_secondaryneighbors.rename(columns={'index':'pair'}).set_index('pair')

### Too big of a file

In [4]:
# filtered_neighbors = pd.DataFrame()
# for flag in attributes.index:
#     taxon = attributes.loc[flag, 'taxon_id']
#     flag_num = attributes.loc[flag, 'num']
#     neighbors.loc[(neighbors['taxon_id']==taxon)&(neighbors['num']>=(flag_num-THRESHOLD))&(neighbors['num']<=(flag_num+THRESHOLD))&(neighbors['family']!='none'), 'flag'] = attributes.loc[flag, 'family_desc']
#     flag_neighbors = neighbors.loc[(neighbors['taxon_id']==taxon)&(neighbors['num']>=(flag_num-THRESHOLD))&(neighbors['num']<=(flag_num+THRESHOLD))&(neighbors['family']!='none')]
#     filtered_neighbors = pd.concat([filtered_neighbors, flag_neighbors], axis=0)

In [5]:
# filtered_neighbors.to_csv('filtered_neighbors.csv')

In [ ]:
# neighbor_list = filtered_neighbors.groupby('taxon_id')['family_desc'].apply(list).reset_index(name="family_desc")
# neighbor_list['pair'] = neighbor_list['family_desc']
# neighbor_list = neighbor_list.explode('family_desc')
# neighbor_list = neighbor_list.explode('pair')

In [ ]:
# top_neighbors = neighbor_list.loc[neighbor_list['family_desc']!=neighbor_list['pair'],['family_desc', 'pair']].value_counts(ascending=False).reset_index(name='count')

### SQL LITE File

In [2]:
import sqlite3
import pandas as pd

In [25]:

# Create a SQL connection to our SQLite database
con = sqlite3.connect(r'data\20230416\SQLITE\25585_96473_230403_IPR018163_43_Cluster5_90_full_ssn_arrow_data_co0_ns10.sqlite')

cur = con.cursor()

# # The result of a "cursor.execute" can be iterated over by row
table = 'neighbors'
# query = "SELECT * FROM sqlite_master WHERE type='table';"
# query = "SELECT * FROM {} LIMIT 5;".format(table)
# query = 'PRAGMA table_info({});'.format(table)
# for row in cur.execute(query):
#     print(row)
    
df = pd.read_sql_query("SELECT * from {}".format(table), con)

# Be sure to close the connection
con.close()

df.to_csv(r'data\20230416\neighbors\neighbors_cluster5.csv')

In [42]:
# df.to_csv('data/cluster_degree.csv')

In [ ]:
# attributes
# sqlite_sequence
# neighbors
# families
# cluster_degree
# metadata
# cluster_index
# uniref50_cluster_index
# uniref50_cluster_index
# uniref50_index
# uniref90_cluster_index
# uniref90_range
# uniref90_index